## Get required SQL Query based on prompt

In [52]:
const user_query = "how many customers work in USA?";

In [37]:
import { OpenAI } from "@langchain/openai";


const openai_api_base='https://delta.helloway.workers.dev/v1';
const openai_api_key= 'sk-BGuzv1Xgj1u5WDObw7sYT3BlbkFJrQd1upRgeTbymcina';


// Initialize a new instance of ChatOpenAI
const llm = new OpenAI({  
    modelName: "gpt-3.5-turbo-0125",
    configuration: {
    baseURL:  openai_api_base,
  },
    openAIApiKey: openai_api_key,
    temperature: 0,
});

prompt_str = `
here is a schema of a 'customers' table saved in an sqlite database. I would like you to understand this table completely,
and understand the following user question about the table and provide any SQL query, that should work for an sqlite database, that would return the 
related records that would answer the user question.
here is the schema:
'{"cid":{"0":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"10":10,"11":11,"12":12},"name":{"0":"CustomerId","1":"FirstName","2":"LastName","3":"Company","4":"Address","5":"City","6":"State","7":"Country","8":"PostalCode","9":"Phone","10":"Fax","11":"Email","12":"SupportRepId"},"type":{"0":"INTEGER","1":"NVARCHAR(40)","2":"NVARCHAR(20)","3":"NVARCHAR(80)","4":"NVARCHAR(70)","5":"NVARCHAR(40)","6":"NVARCHAR(40)","7":"NVARCHAR(40)","8":"NVARCHAR(10)","9":"NVARCHAR(24)","10":"NVARCHAR(24)","11":"NVARCHAR(60)","12":"INTEGER"},"notnull":{"0":1,"1":1,"2":1,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":1,"12":0},"dflt_value":{"0":null,"1":null,"2":null,"3":null,"4":null,"5":null,"6":null,"7":null,"8":null,"9":null,"10":null,"11":null,"12":null},"pk":{"0":1,"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":0,"12":0}}'

user query: {user_query}
please return just the SQL query and nothing else. dont return any extra keywords or explanation.
just return the query
`

const res = await llm.call(prompt_str);


In [38]:
res

SELECT * FROM customers WHERE Country = 'USA';

## Download SQLite database from Github

In [48]:
await import('https://cdn.jsdelivr.net/pyodide/v0.25.0/full/pyodide.js');
console.log('test');
let pyodide = await loadPyodide();
let zipResponse = await fetch("https://github.com/SaiNavyanth2000/public_data/raw/main/hsbc_sample_employee_db.zip");
let zipBinary = await zipResponse.arrayBuffer();
pyodide.unpackArchive(zipBinary, "zip");

test


### Verify if the file is available in pyodide

In [27]:
// To list the files of the root folder in Python
const lst=`import os
print("testing")
def list_files():
    for file_name in os.listdir():
        print(file_name)

list_files()`;

pyodide.runPython(lst);

testing
employees.db
.matplotlib


## Generate and execute required python code to answer the question

### Query the DB to get the related rows in JSON format

In [39]:
// await import('https://cdn.jsdelivr.net/pyodide/v0.25.0/full/pyodide.js');

// let pyodide = await loadPyodide();
await pyodide.loadPackage("micropip");
const micropip = pyodide.pyimport("micropip");
await micropip.install('pandas')
await micropip.install('sqlalchemy')
let extract_data = pyodide.runPython(`
    print('hi')
    from sqlalchemy import text
    import sqlalchemy
    import pandas as pd
    #2.-Turn on database engine
    dbEngine=sqlalchemy.create_engine('sqlite:///employees.db') 
    def extract_data(sql_query):
        with dbEngine.begin() as conn:
            query = text(sql_query)
            df = pd.read_sql_query(query, conn)
            print(df)
            return df.to_json()
    extract_data
  `);
let df_result = extract_data(res)
df_result

micropip already loaded from default channel
No new packages to load
hi
    CustomerId FirstName  ...                     Email SupportRepId
0           16     Frank  ...        fharris@google.com            4
1           17      Jack  ...   jacksmith@microsoft.com            5
2           18  Michelle  ...         michelleb@aol.com            3
3           19       Tim  ...          tgoyer@apple.com            3
4           20       Dan  ...       dmiller@comcast.com            4
5           21     Kathy  ...       kachase@hotmail.com            5
6           22   Heather  ...        hleacock@gmail.com            4
7           23      John  ...    johngordon22@yahoo.com            4
8           24     Frank  ...        fralston@gmail.com            3
9           25    Victor  ...        vstevens@yahoo.com            5
10          26   Richard  ...  ricunningham@hotmail.com            4
11          27   Patrick  ...      patrick.gray@aol.com            4
12          28     Julia  ...  

{"CustomerId":{"0":16,"1":17,"2":18,"3":19,"4":20,"5":21,"6":22,"7":23,"8":24,"9":25,"10":26,"11":27,"12":28},"FirstName":{"0":"Frank","1":"Jack","2":"Michelle","3":"Tim","4":"Dan","5":"Kathy","6":"Heather","7":"John","8":"Frank","9":"Victor","10":"Richard","11":"Patrick","12":"Julia"},"LastName":{"0":"Harris","1":"Smith","2":"Brooks","3":"Goyer","4":"Miller","5":"Chase","6":"Leacock","7":"Gordon","8":"Ralston","9":"Stevens","10":"Cunningham","11":"Gray","12":"Barnett"},"Company":{"0":"Google Inc.","1":"Microsoft Corporation","2":null,"3":"Apple Inc.","4":null,"5":null,"6":null,"7":null,"8":null,"9":null,"10":null,"11":null,"12":null},"Address":{"0":"1600 Amphitheatre Parkway","1":"1 Microsoft Way","2":"627 Broadway","3":"1 Infinite Loop","4":"541 Del Medio Avenue","5":"801 W 4th Street","6":"120 S Orange Ave","7":"69 Salem Street","8":"162 E Superior Street","9":"319 N. Frances Street","10":"2211 W Berry Street","11":"1033 N Park Ave","12":"302 S 700 E"},"City":{"0":"Mountain View","1

In [40]:
typeof df_result

string

### Generate Python code based on the related data and user query

In [55]:
import { OpenAI } from "@langchain/openai";


const openai_api_base='https://delta.helloway.workers.dev/v1';
const openai_api_key= 'sk-BGuzv1Xgj1u5WDObw7sYT3BlbkFJrQd1upRgeTbymcina';


// Initialize a new instance of ChatOpenAI
const llm = new OpenAI({  
    modelName: "gpt-3.5-turbo-0125",
    configuration: {
    baseURL:  openai_api_base,
  },
    openAIApiKey: openai_api_key,
    temperature: 0,
});
prompt_str = `
here is a dataframe in json format:
${df_result}
based of this data above, can you answer this user question:
user query: ${user_query}

if you think any calculation or code is required, just provide the python code to evaluate the answer, and nothing else.
if the output is python code, just generate the code and dont add any extra keyword or explanation to the code. generate a complete executable python code.
include print statements in the code to answer the question as well.
if the output is general text and not code, be the first word as "output" and then the output.
`

const res = await llm.call(prompt_str);


In [47]:
typeof res[0]

string

In [56]:
res

```python
import pandas as pd

data = {
    "CustomerId":{"0":16,"1":17,"2":18,"3":19,"4":20,"5":21,"6":22,"7":23,"8":24,"9":25,"10":26,"11":27,"12":28},
    "FirstName":{"0":"Frank","1":"Jack","2":"Michelle","3":"Tim","4":"Dan","5":"Kathy","6":"Heather","7":"John","8":"Frank","9":"Victor","10":"Richard","11":"Patrick","12":"Julia"},
    "LastName":{"0":"Harris","1":"Smith","2":"Brooks","3":"Goyer","4":"Miller","5":"Chase","6":"Leacock","7":"Gordon","8":"Ralston","9":"Stevens","10":"Cunningham","11":"Gray","12":"Barnett"},
    "Company":{"0":"Google Inc.","1":"Microsoft Corporation","2":None,"3":"Apple Inc.","4":None,"5":None,"6":None,"7":None,"8":None,"9":None,"10":None,"11":None,"12":None},
    "Address":{"0":"1600 Amphitheatre Parkway","1":"1 Microsoft Way","2":"627 Broadway","3":"1 Infinite Loop","4":"541 Del Medio Avenue","5":"801 W 4th Street","6":"120 S Orange Ave","7":"69 Salem Street","8":"162 E Superior Street","9":"319 N. Frances Street","10":"2211 W Berry Street","11":"1033

In [57]:
// Remove the quotes and 'Python' keyword
const code = res.replace("python","").replace(/`{3}/g, "");

In [58]:
code


import pandas as pd

data = {
    "CustomerId":{"0":16,"1":17,"2":18,"3":19,"4":20,"5":21,"6":22,"7":23,"8":24,"9":25,"10":26,"11":27,"12":28},
    "FirstName":{"0":"Frank","1":"Jack","2":"Michelle","3":"Tim","4":"Dan","5":"Kathy","6":"Heather","7":"John","8":"Frank","9":"Victor","10":"Richard","11":"Patrick","12":"Julia"},
    "LastName":{"0":"Harris","1":"Smith","2":"Brooks","3":"Goyer","4":"Miller","5":"Chase","6":"Leacock","7":"Gordon","8":"Ralston","9":"Stevens","10":"Cunningham","11":"Gray","12":"Barnett"},
    "Company":{"0":"Google Inc.","1":"Microsoft Corporation","2":None,"3":"Apple Inc.","4":None,"5":None,"6":None,"7":None,"8":None,"9":None,"10":None,"11":None,"12":None},
    "Address":{"0":"1600 Amphitheatre Parkway","1":"1 Microsoft Way","2":"627 Broadway","3":"1 Infinite Loop","4":"541 Del Medio Avenue","5":"801 W 4th Street","6":"120 S Orange Ave","7":"69 Salem Street","8":"162 E Superior Street","9":"319 N. Frances Street","10":"2211 W Berry Street","11":"1033 N Park A

### Execute the clean Python code

In [45]:
// await import('https://cdn.jsdelivr.net/pyodide/v0.25.0/full/pyodide.js');

// let pyodide = await loadPyodide();
await pyodide.loadPackage("micropip");
const micropip = pyodide.pyimport("micropip");
await micropip.install('pandas')
await micropip.install('matplotlib')
let extract_data = pyodide.runPython(code);


micropip already loaded from default channel
No new packages to load
Number of users living in USA: 13


## Helper code to generate schema of a database

In [50]:
// await import('https://cdn.jsdelivr.net/pyodide/v0.25.0/full/pyodide.js');
// console.log('test');
// let pyodide = await loadPyodide();
// let zipResponse = await fetch("https://github.com/SaiNavyanth2000/public_data/raw/main/hsbc_sample_employee_db.zip");
// let zipBinary = await zipResponse.arrayBuffer();
// pyodide.unpackArchive(zipBinary, "zip");
await pyodide.loadPackage("micropip");
const micropip = pyodide.pyimport("micropip");
await micropip.install('pandas')
await micropip.install('sqlalchemy')
let extract_data = pyodide.runPython(`
from sqlalchemy import text
import sqlalchemy
import pandas as pd
#2.-Turn on database engine
dbEngine=sqlalchemy.create_engine('sqlite:///employees.db', echo=True) 
def extract_data(sql_query):
    with dbEngine.begin() as conn:
        query = text(sql_query)
        df = pd.read_sql_query(query, conn)
        return df.to_json()
extract_data
`);

    
schema_str = extract_data("pragma table_info(customers)")

micropip already loaded from default channel
No new packages to load
2024-06-12 15:36:59,048 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-12 15:36:59,049 INFO sqlalchemy.engine.Engine pragma table_info(customers)
2024-06-12 15:36:59,052 INFO sqlalchemy.engine.Engine [generated in 0.00200s] ()
2024-06-12 15:36:59,067 INFO sqlalchemy.engine.Engine COMMIT


In [51]:
schema_str

{"cid":{"0":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"10":10,"11":11,"12":12},"name":{"0":"CustomerId","1":"FirstName","2":"LastName","3":"Company","4":"Address","5":"City","6":"State","7":"Country","8":"PostalCode","9":"Phone","10":"Fax","11":"Email","12":"SupportRepId"},"type":{"0":"INTEGER","1":"NVARCHAR(40)","2":"NVARCHAR(20)","3":"NVARCHAR(80)","4":"NVARCHAR(70)","5":"NVARCHAR(40)","6":"NVARCHAR(40)","7":"NVARCHAR(40)","8":"NVARCHAR(10)","9":"NVARCHAR(24)","10":"NVARCHAR(24)","11":"NVARCHAR(60)","12":"INTEGER"},"notnull":{"0":1,"1":1,"2":1,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":1,"12":0},"dflt_value":{"0":null,"1":null,"2":null,"3":null,"4":null,"5":null,"6":null,"7":null,"8":null,"9":null,"10":null,"11":null,"12":null},"pk":{"0":1,"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":0,"12":0}}